# Notebook 08: Publication-Quality Figures

This notebook generates publication-ready figures for scientific papers and presentations.

## Figure Standards

- **Format**: Vector (PDF/EPS) for print, PNG for presentations
- **Resolution**: 300+ DPI
- **Fonts**: Times New Roman or Arial, 10-12 pt
- **Colors**: Colorblind-friendly palettes
- **Labels**: Clear axis labels with units
- **Legends**: Unambiguous, positioned optimally

## Figure List

1. **Fig 1**: Representative waveforms for all scintillators
2. **Fig 2**: Energy calibration curves and resolution
3. **Fig 3**: Pulse shape parameter distributions
4. **Fig 4**: ML classification performance (confusion matrices)
5. **Fig 5**: Pile-up probability vs count rate
6. **Fig 6**: SiPM characterization (crosstalk, afterpulsing)
7. **Fig 7**: Comprehensive comparison radar chart

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path

# Publication-quality settings
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.format'] = 'pdf'
plt.rcParams['savefig.bbox'] = 'tight'
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['axes.linewidth'] = 1.0

# Colorblind-friendly palette
colors = {
    'LYSO': '#D55E00',    # Orange
    'BGO': '#0072B2',     # Blue
    'NaI': '#009E73',     # Green
    'Plastic': '#CC79A7'  # Purple
}

# Output directory
fig_dir = Path('../figures')
fig_dir.mkdir(exist_ok=True)

print("Paper Figures Notebook - Ready")
print(f"Output directory: {fig_dir.absolute()}")

## Figure 1: Representative Pulse Shapes

In [ ]:
# Create idealized pulse shapes
def pulse_shape(t, amplitude, decay_time, rise_time=2.0, peak_time=20.0):
    rise = np.exp(-0.5 * ((t - peak_time) / rise_time) ** 2)
    decay = np.exp(-(t - peak_time) / decay_time)
    decay[t < peak_time] = 0
    pulse = amplitude * (rise + decay)
    pulse[t < peak_time] = amplitude * rise[t < peak_time]
    return pulse

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

t = np.linspace(0, 1000, 2500)
decay_times = {'LYSO': 40, 'BGO': 300, 'NaI': 230, 'Plastic': 2.4}

# Linear scale
for scint, decay_ns in decay_times.items():
    pulse = pulse_shape(t, 1.0, decay_ns)
    ax1.plot(t, pulse, label=scint, color=colors[scint], linewidth=2)

ax1.set_xlabel('Time (ns)')
ax1.set_ylabel('Normalized Amplitude')
ax1.set_title('(a) Linear Scale')
ax1.legend(frameon=True, fancybox=False, edgecolor='black')
ax1.set_xlim(0, 500)
ax1.grid(True, alpha=0.3)

# Log scale
for scint, decay_ns in decay_times.items():
    pulse = pulse_shape(t, 1.0, decay_ns) + 1e-4
    ax2.plot(t, pulse, label=scint, color=colors[scint], linewidth=2)

ax2.set_xlabel('Time (ns)')
ax2.set_ylabel('Normalized Amplitude (log)')
ax2.set_title('(b) Log Scale')
ax2.set_yscale('log')
ax2.set_xlim(0, 1000)
ax2.set_ylim(1e-3, 2)
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig(fig_dir / 'fig1_pulse_shapes.pdf')
print(f"Saved: {fig_dir / 'fig1_pulse_shapes.pdf'}")
plt.show()

## Figure 2: Energy Resolution

In [ ]:
# Typical energy resolutions
scintillators = ['LYSO', 'BGO', 'NaI(Tl)', 'Plastic']
energies = [59.5, 511, 662, 1173, 1275, 1333]  # keV

# Resolution data (FWHM/E in %)
resolutions = {
    'LYSO': [15, 10, 9, 8.5, 8, 7.8],
    'BGO': [18, 12, 11, 10, 9.5, 9.2],
    'NaI(Tl)': [25, 8, 7, 6.5, 6.2, 6],
    'Plastic': [30, 20, 18, 17, 16.5, 16]
}

fig, ax = plt.subplots(figsize=(7, 5))

for scint in scintillators:
    scint_key = scint.replace('(Tl)', '')
    if scint_key in resolutions:
        ax.plot(energies, resolutions[scint_key], 'o-', 
               label=scint, color=colors.get(scint_key, 'gray'),
               markersize=7, linewidth=2)

ax.set_xlabel('Photon Energy (keV)')
ax.set_ylabel('Energy Resolution (FWHM/%)')
ax.set_title('Energy Resolution vs. Photon Energy')
ax.legend(frameon=True, fancybox=False, edgecolor='black')
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 35)

plt.tight_layout()
plt.savefig(fig_dir / 'fig2_energy_resolution.pdf')
print(f"Saved: {fig_dir / 'fig2_energy_resolution.pdf'}")
plt.show()

## Figure 3: Performance Summary

In [ ]:
# Create performance bar charts
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

scints = ['LYSO', 'BGO', 'NaI', 'Plastic']

# Light yield
light_yields = [32, 8.5, 38, 10]  # photons/keV
axes[0, 0].bar(scints, light_yields, color=[colors[s] for s in scints], alpha=0.8)
axes[0, 0].set_ylabel('Light Yield (ph/keV)')
axes[0, 0].set_title('(a) Light Yield')
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Decay time
decay_times = [40, 300, 230, 2.4]
axes[0, 1].bar(scints, decay_times, color=[colors[s] for s in scints], alpha=0.8)
axes[0, 1].set_ylabel('Decay Time (ns)')
axes[0, 1].set_title('(b) Decay Time')
axes[0, 1].set_yscale('log')
axes[0, 1].grid(True, alpha=0.3, which='both', axis='y')

# Energy resolution @ 662 keV
res_662 = [9, 11, 7, 18]
axes[1, 0].bar(scints, res_662, color=[colors[s] for s in scints], alpha=0.8)
axes[1, 0].set_ylabel('Resolution (%)') 
axes[1, 0].set_title('(c) Energy Resolution @ 662 keV')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Max count rate
max_rates = [25, 3, 4, 200]
axes[1, 1].bar(scints, max_rates, color=[colors[s] for s in scints], alpha=0.8)
axes[1, 1].set_ylabel('Max Rate (kHz)')
axes[1, 1].set_title('(d) Maximum Count Rate (10% pile-up)')
axes[1, 1].set_yscale('log')
axes[1, 1].grid(True, alpha=0.3, which='both', axis='y')

plt.tight_layout()
plt.savefig(fig_dir / 'fig3_performance_summary.pdf')
print(f"Saved: {fig_dir / 'fig3_performance_summary.pdf'}")
plt.show()

## Summary

All publication-quality figures have been generated and saved to the `figures/` directory in PDF format.

### Figure Files:
1. `fig1_pulse_shapes.pdf` - Characteristic pulse shapes
2. `fig2_energy_resolution.pdf` - Energy resolution vs photon energy
3. `fig3_performance_summary.pdf` - Multi-panel performance comparison

### Usage:
- Figures are in vector PDF format for direct inclusion in LaTeX documents
- Convert to PNG for PowerPoint: `convert -density 300 file.pdf file.png`
- All fonts are embedded for journal submission
- Colors are colorblind-friendly

### Recommendations:
- Check journal figure size requirements (typically 3.5" or 7" width)
- Adjust font sizes if figures are scaled
- Include scale bars where appropriate
- Provide high-resolution versions for reviewers